In [ ]:
import numpy as np
from typing import Tuple, Callable

In [ ]:
MATCH = 1
MISMATCH = -1
INDEL = -1

In [ ]:
def print_align(func: Callable[[str, str], Tuple[str, str, float]]):
    """
    decorator for printing an alignment
    | - matched bases
    * - mismatched bases
    """
    def wrapper(seq1, seq2):
        aligned_seq1, aligned_seq2, score = func(seq1, seq2)
        alignment = ""
        for i, j in zip(aligned_seq1, aligned_seq2):
            if i.islower() or j.islower() or i == "-" or j == "-":
                alignment += " "
            elif i == j:
                alignment += "|"
            elif i != j:
                alignment += "*"
        print(f"{aligned_seq1}\n{alignment}\n{aligned_seq2}")
        print(f"Alignment score: {score}")
    return wrapper

@print_align
def smith_waterman(seq1: str, seq2: str) -> Tuple[str, str, float]:
    """
    Performs local alignment of seq1 and seq2 using Smith-Waterman algorithm
    with linear gap penalty.
    MATCH, MISMATCH and INDEL scores are taken from global namespace
    Returns: aligned_seq1, aligned_seq2
    Uppercase letters - aligned regions
    Lowercase letters - unaligned regions
    '-' - indels
    """
    # Initialize dynamic programming graph
    n = len(seq1) + 1
    m = len(seq2) + 1
    dp = np.zeros((n, m))

    # backtracking matrix
    pi = np.array([[None] * m for _ in range(n)])

    # Filling in the matrix using Smith-Waterman general recursion
    for i in range(1, n):
        for j in range(1, m):
            # dict, that contains three variants(match, del1, del2)
            d = {}
            d[(i - 1, j - 1)] = dp[i - 1, j - 1] + (MATCH if seq1[i - 1] == seq2[j - 1] else MISMATCH)
            d[(i - 1, j)] = dp[i - 1, j] + INDEL
            d[(i, j - 1)] = dp[i, j - 1] + INDEL
            # for local alignment
            d[0] = 0
            dp[i, j] = max(d.values())
            pi[i, j] = max(d, key = d.get)

    # Backtracking
    aligned_seq1 = ""
    aligned_seq2 = ""

    ind = np.unravel_index(np.argmax(dp, axis=None), dp.shape)
    # right boundaries of aligned regions in seq1 and seq1
    rights = ind
    # score of the alignment
    align_score = dp[ind]

    # performs a reverse pass through dp matrix to build an alignment
    i_, j_ = ind
    score = align_score
    while score > 0:
        i, j = pi[i_, j_]
        if i == i_ - 1 and j == j_ - 1:
            aligned_seq1 += seq1[i]
            aligned_seq2 += seq2[j]
        elif i == i_  and j == j_ - 1:
            aligned_seq1 += "-"
            aligned_seq2 += seq2[j]
        else:
            aligned_seq1 += seq1[i]
            aligned_seq2 += "-"
        score = dp[i, j]
        i_, j_ = i, j

    # left boundaries of aligned regions in seq1 and seq1
    lefts = i, j

    aligned_seq1 = aligned_seq1[::-1]
    aligned_seq2 = aligned_seq2[::-1]

    # add unaligned regions

    # left part
    aligned_seq1 = "-" * (max(lefts) - lefts[0]) + seq1[:lefts[0]].lower() + aligned_seq1
    aligned_seq2 = "-" * (max(lefts) - lefts[1]) + seq2[:lefts[1]].lower() + aligned_seq2

    # right part
    aligned_seq1 += seq1[rights[0]:].lower()
    aligned_seq2 += seq2[rights[1]:].lower()
    aligned_seq1 = aligned_seq1.ljust(max(len(aligned_seq1), len(aligned_seq2)), "-")
    aligned_seq2 = aligned_seq2.ljust(max(len(aligned_seq1), len(aligned_seq2)), "-")

    return aligned_seq1, aligned_seq2, align_score

# Testing

Так как в данной реализации алгоритма матрица замен не используется, не имеет значения к какому типу последовательностей (белковые, нуклеиновые или произвольные) он будет применён

В выравниваниях заглавные буквы означают выравненные участки, строчные буквы - невыравненные участки.  
| - совпадающие буквы  
\* - не совпадающие

Для начала возьмем полностью идентичные последовательности:

In [ ]:
seq1 = "QWERTYQWERTY"
seq2 = "QWERTYQWERTY"
smith_waterman(seq1, seq2)

QWERTYQWERTY
||||||||||||
QWERTYQWERTY
Alignment score: 12.0


Добавим по краям невыравниваемые последовательности

In [ ]:
seq1 = "AAAAAAAAAAQWERTYQWERTYGGGGGGG"
seq2 = "TTTTTTQWERTYQWERTYCCCCCCCCCCC"
smith_waterman(seq1, seq2)

aaaaaaaaaaQWERTYQWERTYggggggg----
          ||||||||||||           
----ttttttQWERTYQWERTYccccccccccc
Alignment score: 12.0


Внесём замены в гомологичные участки:  
QWERT**P**QWE**N**TY  
QW**Y**RTYQWERTY

In [ ]:
seq1 = "AAAAAAAAAAQWERTPQWENTYGGGGGGG"
seq2 = "TTTTTTQWYRTYQWERTYC"
smith_waterman(seq1, seq2)

aaaaaaaaaaQWERTPQWENTYggggggg
          ||*||*|||*||       
----ttttttQWYRTYQWERTYc------
Alignment score: 6.0


Добавим короткие индели:  
QW**R**RT**P**YQW**--**ERTY  
QWERT**-**YQW**AA**ERTY

In [ ]:
seq1 = "AAAAAAAAAAQWRRTPYQWERTYGGGGGGG"
seq2 = "TTTTTTQWERTYQWAAERTYCCCCCCCCCCC"
smith_waterman(seq1, seq2)

aaaaaaaaaaQWRRTPYQW--ERTYggggggg----
          ||*|| |||  ||||           
----ttttttQWERT-YQWAAERTYccccccccccc
Alignment score: 7.0


При этом если добавить слишком длинные инсерции(или сликшом много замен), то алгоритм с линейными штрафами за делеции не будет их обнаруживать, так как суммарный штраф за делецию начнёт превышать балл за выравнивание второго гомологичного участка:

In [ ]:
seq1 = "QWERTYAAAAAQWERTY" # 5A
seq2 = "QWERTYQWERTY"
smith_waterman(seq1, seq2)

QWERTYAAAAAQWERTY
||||||     ||||||
QWERTY-----QWERTY
Alignment score: 7.0


In [ ]:
seq1 = "QWERTYAAAAAAQWERTY" # 6A
seq2 = "QWERTYQWERTY"
smith_waterman(seq1, seq2)

QWERTYaaaaaaqwerty
||||||            
QWERTYqwerty------
Alignment score: 6.0


Если добавить слишком много замен(даже на синонимичные остатки: Q-N, D-E и т.п.), тоже ломается ==> лучше использовать матрицу замен

In [ ]:
seq1 = "QWDRTYNWEKTT"
seq2 = "NWERPYQWERTY"
smith_waterman(seq1, seq2)

------QWDRTYnwektt
      ||*|||      
nwerpyQWERTY------
Alignment score: 4.0


Попробуем применить данный алгоритм к реальным последовательностям из UniProt на бета-субъединицы гемоглобина для:  
человека: P68871  
мыши: P02088    
быка: P02070  
свиньи: P02067   
курицы: P02112

In [ ]:
homo = "MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH"
mouse = "MVHLTDAEKAAVSCLWGKVNSDEVGGEALGRLLVVYPWTQRYFDSFGDLSSASAIMGNAKVKAHGKKVITAFNDGLNHLDSLKGTFASLSELHCDKLHVDPENFRLLGNMIVIVLGHHLGKDFTPAAQAAFQKVVAGVATALAHKYH"
bovine ="MLTAEEKAAVTAFWGKVKVDEVGGEALGRLLVVYPWTQRFFESFGDLSTADAVMNNPKVKAHGKKVLDSFSNGMKHLDDLKGTFAALSELHCDKLHVDPENFKLLGNVLVVVLARNFGKEFTPVLQADFQKVVAGVANALAHRYH"
pig = "MVHLSAEEKEAVLGLWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSNADAVMGNPKVKAHGKKVLQSFSDGLKHLDNLKGTFAKLSELHCDQLHVDPENFRLLGNVIVVVLARRLGHDFNPNVQAAFQKVVAGVANALAHKYH"
chick = "MVHWTAEEKQLITGLWGKVNVAECGAEALARLLIVYPWTQRFFASFGNLSSPTAILGNPMVRAHGKKVLTSFGDAVKNLDNIKNTFSQLSELHCDKLHVDPENFRLLGDILIIVLAAHFSKDFTPECQAAWQKLVRVVAHALARKYH"

In [ ]:
smith_waterman(homo, mouse)

MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH
|||||**||*||**||||||*||||||||||||||||||||*|*||||||***|*|||*|||||||||**||*|||*|||*||||||*|||||||||||||||||||||**|*||*||*||*|||**|||*||||||||*|||||||
MVHLTDAEKAAVSCLWGKVNSDEVGGEALGRLLVVYPWTQRYFDSFGDLSSASAIMGNAKVKAHGKKVITAFNDGLNHLDSLKGTFASLSELHCDKLHVDPENFRLLGNMIVIVLGHHLGKDFTPAAQAAFQKVVAGVATALAHKYH
Alignment score: 89.0


In [ ]:
smith_waterman(homo, bovine)

mvhLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPV-QAAYQKVVAGVANALAHKYH
   ||*|||*||||*||||*|||||||||||||||||||||||||||||||*||||*||||||||||||**||*|**|||*||||||*||||||||||||||||*|||||||*|||**|||||| || ||**|||||||||||||*||
--mLTAEEKAAVTAFWGKVKVDEVGGEALGRLLVVYPWTQRFFESFGDLSTADAVMNNPKVKAHGKKVLDSFSNGMKHLDDLKGTFAALSELHCDKLHVDPENFKLLGNVLVVVLARNFGKEFT-PVLQADFQKVVAGVANALAHRYH
Alignment score: 101.0


In [ ]:
smith_waterman(homo, chick)

MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH
|||*|*|||***|*|||||||*|*|*|||*|||*|||||||||*|||*||*|*|**|||*|*|||||||**|*|****|||*|*||**||||||||||||||||||||**|**|||*||*|*|||**|||*||*|**||*|||*|||
MVHWTAEEKQLITGLWGKVNVAECGAEALARLLIVYPWTQRFFASFGNLSSPTAILGNPMVRAHGKKVLTSFGDAVKNLDNIKNTFSQLSELHCDKLHVDPENFRLLGDILIIVLAAHFSKDFTPECQAAWQKLVRVVAHALARKYH
Alignment score: 57.0


Видим, что алгоритм находит гомологичные участки в близких последовательностях, однако гэпы расставлены "в рассыпную", а мы ожидаем, что они будут группироваться ==> нужен более сложный алгоритм с афинными штрафами за гэпы и с матрицей замен вместо match/mismatch